In [34]:
timestamp = re.search(re.compile('\d{2}:\d{2}'),
                          'real_time_msrb_file_2023-06-21_14:09:02.json')[0]
    
    #the timestamp on a file received can never be larger than the current time, because such a file does not exist 
    #it must hence be the same time or earlier, and if too long ago we exit the function
if int(timestamp[:2]) != now.hour and now.minute - int(timestamp[2:]) >= 2:
    print(f'{name} is asynchronous from time now, {now}. Exiting function.')

In [2]:
from cloudevents.http import CloudEvent
from cloudevents.conversion import to_binary
import requests

attributes = {
    "type": "com.example.sampletype1",
    "source": "https://example.com/event-producer",
}


def post_cloud_event(bucket, name):
    # data = {'bucket':"msrb_intraday_real_time_trade_files",
    #         "name" : 'real_time_msrb_file_2023-06-14_15:00:01.json'
    #        }
    
    data = {'bucket': bucket, 'name':name}
    event = CloudEvent(attributes, data)

    # Creates the HTTP request representation of the CloudEvent in binary content mode
    headers, body = to_binary(event)

    # POST
    print(f'Posting {data}')
    requests.post('http://localhost:5000', data=body, headers=headers)

In [2]:
from auxiliary_functions import *

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/isaac_creds.json"
fs = gcsfs.GCSFileSystem(project='eng-reactor-287421')

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [97]:
bias_path = "gs://biases/bias_2023-06-15.pkl"
bias = load_from_cloud_storage(bias_path, fs)
len(bias)

3495

In [6]:
%time test_files = get_all_msrb_files('2023-06-20')

CPU times: user 9.55 s, sys: 377 ms, total: 9.93 s
Wall time: 45.3 s


In [9]:
sorted(test_files)[-10:]

['msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:19:01.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:20:01.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:21:01.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:22:02.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:23:01.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:24:02.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:25:02.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:26:02.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:27:02.json',
 'msrb_intraday_real_time_trade_files/real_time_msrb_file_2023-06-20_18:28:02.json']

Try processing 50 trade files:

In [6]:
post_cloud_event("msrb_intraday_real_time_trade_files", 'real_time_msrb_file_2023-06-28_14:02:02.json')

Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-28_14:02:02.json'}


In [ ]:
for file in [s.split('/')[1] for s in test_files[250:300]]: 
    post_cloud_event("msrb_intraday_real_time_trade_files", file)

Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:10:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:11:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:12:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:13:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:14:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:15:01.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:16:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:17:02.json'}
Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2

Send a non-existent file:

In [3]:
post_cloud_event("msrb_intraday_real_time_trade_files", 'notfile.pkl')

Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'notfile.pkl'}


Send an existing file but where the file has no trades:

In [106]:
 with fs.open('msrb_intraday_real_time_trade_files/', 'rb') as f:
            trade_messages = json.load(f)
print(trade_messages)

[]


In [12]:
post_cloud_event("msrb_intraday_real_time_trade_files", 'real_time_msrb_file_2023-06-16_10:04:02.json')

Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-16_10:04:02.json'}


Post an old file

In [108]:
bias_path = "gs://biases/bias_2023-06-15.pkl"
bias = load_from_cloud_storage(bias_path, fs)

In [110]:
bias[-5:]

[-0.03082969390474155,
 -0.0308340585942173,
 -0.030866407188323834,
 -0.030877299368012253,
 -0.03086485419688465]

In [121]:
post_cloud_event("msrb_intraday_real_time_trade_files", test_files[274].split('/')[1])

Posting {'bucket': 'msrb_intraday_real_time_trade_files', 'name': 'real_time_msrb_file_2023-06-15_09:34:02.json'}


In [115]:
bias_path = "gs://biases/bias_2023-06-15.pkl"
biasafter = load_from_cloud_storage(bias_path, fs)

In [117]:
bias[-5:] == biasafter[-5:]

True